* Issue [37](https://github.com/salgo60/SCB-Wikidata/issues/37)
* denna Notebook [SCB_37_myndighetsregister.ipynb](https://github.com/salgo60/SCB-Wikidata/blob/main/notebook/SCB_37_myndighetsregister.ipynb)

In [1]:
import time

from datetime import datetime

now = datetime.now()
timestamp = now.timestamp()

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

Start: 2025-11-20 13:03:52


In [13]:
# Notebook: Myndighetsregistret -> Wikidata matching
# File source (local): /mnt/data/Statliga förvaltningsmyndigheter.txt

# Syfte:
# 1. Läs in SCB:s nedladdade fil (Statliga förvaltningsmyndigheter)
# 2. Normalisera organisationsnummer
# 3. Fråga Wikidata (P6460 Swedish Organization Number) för matchande poster
# 4. Jämför värden (namn, webb, epost etc.) och markera avvikelser
# 5. Skriv ut rapporter (CSV/JSON)

# Instruktion: kör cellerna i en Jupyter Notebook-miljö (eller kör som script). Alla paths är relativa till körmiljön.

# --- Installera beroenden (avkommentera vid behov) ---
# !pip install pandas requests tqdm SPARQLWrapper

import pandas as pd
import re
import requests
import time
from SPARQLWrapper import SPARQLWrapper, JSON
from tqdm import tqdm

# Organisationsnumret är det primära ID:t i myndighetsregistret och motsvarar direkt Wikidata-property wdt:P6460

# --- 1. Läs in SCB-filen ---
SCB_FILE = 'myndighetsreg/Statliga förvaltningsmyndigheter.txt'  # <- lokal fil som du laddat upp

# Filen ser ut att vara tab-separerad med en header-rad
df = pd.read_csv(SCB_FILE, sep='\t', dtype=str, encoding='utf-8')
print(f"Läste in {len(df)} rader från SCB-filen")

# Rensa: standardisera kolumnnamn (trimma och gör enklare namn)
orig_cols = df.columns.tolist()
colmap = {c: c.strip() for c in orig_cols}
df.rename(columns=colmap, inplace=True)

# Visa kolumner
print('Kolumner:', list(df.columns))

# --- 2. Normalisera organisationsnummer ---
# Normalisera organisationsnummer till formatet YYYYMM-NNNN (krävs för Wikidata P6460)

def normalize_orgnr(v):
    if pd.isna(v): return None
    s = str(v).strip().replace(' ', '')
    digits = re.sub(r"[^0-9]", "", s)
    if len(digits) == 10:
        return digits[:6] + '-' + digits[6:]
    return s

df['orgnr_norm'] = df['Organisationsnr'].apply(normalize_orgnr)

# --- 3. Hämta matchningar från Wikidata --- Hämta matchningar från Wikidata ---
# Wikidata property för Swedish Organization Number är P6460
SPARQL_ENDPOINT = 'https://query.wikidata.org/sparql'

sparql = SPARQLWrapper(SPARQL_ENDPOINT)
sparql.setReturnFormat(JSON)

# Funktion som frågar Wikidata med en chunk av organisationsnummer (plain, utan bindestreck) 
def query_wikidata_for_orgnrs(orgnr_list):
    if not orgnr_list:
        return {}

    vals = " ".join(f'"{o}"' for o in orgnr_list)

    q = """
    SELECT ?item ?itemLabel ?orgnr ?site ?website WHERE {
      VALUES ?orgnr { %s }
      ?item wdt:P6460 ?orgnr .
      OPTIONAL { ?item wdt:P856 ?website . }
      OPTIONAL { ?item wdt:P973 ?site . }
      SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
    }
    """ % vals

    sparql.setQuery(q)
    ...

    # följsamhet: sätt User-Agent
    sparql.addCustomHttpHeader('User-Agent', 'SCB-Wikidata-Matcher/1.0 (your@email.example)')
    res = sparql.query().convert()
    out = {}
    for b in res['results']['bindings']:
        org = b['orgnr']['value']
        item = b['item']['value']
        label = b.get('itemLabel', {}).get('value')
        website = b.get('website', {}).get('value')
        site = b.get('site', {}).get('value')
        out[re.sub(r'[^0-9]', '', org)] = {
            'wikidata_item': item,
            'label': label,
            'website': website,
            'site': site
        }
    return out

# Kör i batchar (Wikidata SPARQL kan hantera ett rimligt antal VALUES — kör t.ex. 50 åt gången)
#orgnrs = df['orgnr_norm'].dropna().unique().tolist().dropna().unique().tolist()

orgnrs = (
    df['orgnr_norm']
    .dropna()
    .unique()
    .tolist()
)

print(f"Unika orgnummer: {len(orgnrs)}")

batch_size = 50
wikidata_map = {}
for i in tqdm(range(0, len(orgnrs), batch_size), desc='Hämtar från Wikidata'):
    chunk = orgnrs[i:i+batch_size]
    try:
        res = query_wikidata_for_orgnrs(chunk)
        wikidata_map.update(res)
    except Exception as e:
        print('SPARQL-fel vid chunk', i, e)
        time.sleep(5)

print(f"Matchade {len(wikidata_map)} organisationsnummer i Wikidata")

# --- 4. Slå ihop data och hitta avvikelser ---
# Lägg in matchningskolumner i df

def lookup_wikidata(plain):
    if pd.isna(plain):
        return None
    return wikidata_map.get(plain, None)

df['wikidata'] = df['orgnr_norm'].apply(lambda x: lookup_wikidata(x))

# Extrahera kolumner från wikidata-dict

df['wd_item'] = df['wikidata'].apply(lambda x: x['wikidata_item'] if isinstance(x, dict) else None)
df['wd_label'] = df['wikidata'].apply(lambda x: x.get('label') if isinstance(x, dict) else None)
df['wd_website'] = df['wikidata'].apply(lambda x: x.get('website') if isinstance(x, dict) else None)

def compare_text(a, b):
    if pd.isna(a) and pd.isna(b):
        return False
    if pd.isna(a) != pd.isna(b):
        return True
    a_s = str(a).strip().lower()
    b_s = str(b).strip().lower()
    return a_s != b_s

# Avvikelse: namn skiljer sig

df['avvik_namn'] = df.apply(lambda r: compare_text(r['Namn'], r['wd_label']), axis=1)
# Avvikelse: webb skiljer sig (kontrollera delvis match eller olika domän)

def website_normalize(u):
    if pd.isna(u):
        return None
    u = str(u).strip()
    u = re.sub(r'^https?://', '', u)
    u = u.rstrip('/')
    return u.lower()

df['post_webb'] = df['Webbadress'].apply(lambda x: website_normalize(x) if 'Webbadress' in df.columns else None)
df['wd_webb_norm'] = df['wd_website'].apply(website_normalize)
df['avvik_webb'] = df.apply(lambda r: compare_text(r['post_webb'], r['wd_webb_norm']), axis=1)

# Markera poster utan match i Wikidata
ndf = df.copy()
ndf['matched_in_wikidata'] = ndf['wd_item'].notna()

# --- 5. Dubbelriktad kontroll (SCB → WD och WD → SCB) ---
# 5a. SCB → Wikidata: frågor vi redan gjort
# 5b. Wikidata → SCB: hämta alla Wikidata‑poster som har P6460 och jämföra mot SCB

# Hämta ALLa Wikidata‑poster med P6460 (kan vara många, hämta i batch om nödvändigt)

sparql_all = SPARQLWrapper(SPARQL_ENDPOINT)
sparql_all.setReturnFormat(JSON)
q_all = """
SELECT ?item ?itemLabel ?orgnr WHERE {
  ?item wdt:P6460 ?orgnr .
  SERVICE wikibase:label { bd:serviceParam wikibase:language 'sv,en'. }
}
"""

sparql_all.setQuery(q_all)
sparql_all.addCustomHttpHeader('User-Agent', 'SCB-Wikidata-Matcher/1.0')
res_all = sparql_all.query().convert()

wikidata_full = {}
for b in res_all['results']['bindings']:
    org = b['orgnr']['value']
    item = b['item']['value']
    label = b.get('itemLabel', {}).get('value')
    wikidata_full[re.sub(r'[^0-9]', '', org)] = {
        'item': item,
        'label': label
    }

# Lista: de som finns i Wikidata men inte i SCB
set_scb = set(df['orgnr_norm'].dropna())
set_wd = set(wikidata_full.keys())
wd_not_in_scb = sorted(list(set_wd - set_scb))

print(f"Poster i Wikidata men saknas i SCB: {len(wd_not_in_scb)}")

wd_missing_df = pd.DataFrame([
    {
        'orgnr': o,
        'wikidata_item': wikidata_full[o]['item'],
        'wikidata_label': wikidata_full[o]['label']
    } for o in wd_not_in_scb
])

# --- 6. Rapporter (med tidsstämplade filnamn) ---
from datetime import datetime
TS = datetime.now().strftime('%y%m%d_%H%M%S')

out_all = f'myndighetsreg/MR_enriched_{TS}.csv'
out_nomatch = f'myndighetsreg/MR_no_match_{TS}.csv'
out_discrep = f'myndighetsreg/MR_discrepancies_{TS}.csv'
out_wd_missing = f'myndighetsreg/Wikidata_not_in_SCB_{TS}.csv'

ndf.to_csv(out_all, index=False)
ndf[~ndf['matched_in_wikidata']].to_csv(out_nomatch, index=False)
ndf[ndf['avvik_namn'] | ndf['avvik_webb']].to_csv(out_discrep, index=False)
wd_missing_df.to_csv(out_wd_missing, index=False)

print('Filer skapade:')
print(out_all)
print(out_nomatch)
print(out_discrep)
print(out_wd_missing)

# Slut
# Skriv ut sammanfattning
print('Totalt poster:', len(ndf))
print('Hittade i Wikidata:', ndf['matched_in_wikidata'].sum())
print('Namn-avvikelser:', ndf['avvik_namn'].sum())
print('Webb-avvikelser:', ndf['avvik_webb'].sum())
print('Ej matchade i Wikidata:', (~ndf['matched_in_wikidata']).sum())

# Spara rapporter
ndf.to_csv('myndighetsregistret_enriched.csv', index=False)
ndf[~ndf['matched_in_wikidata']].to_csv('myndighetsregistret_no_wikidata_match.csv', index=False)
ndf[ndf['avvik_namn'] | ndf['avvik_webb']].to_csv('myndighetsregistret_discrepancies.csv', index=False)

print('Filer sparade: myndighetsregistret_enriched.csv, myndighetsregistret_no_wikidata_match.csv, myndighetsregistret_discrepancies.csv')

# --- Extra: Förslag på nästa steg ---
# - Kör en fuzzy-matchning (fuzzywuzzy / rapidfuzz) mot Wikidata-labels för ej matchade organisationer
# - Hämta fler fält från Wikidata (ex. official website, country, inception, dissolved) för att bättre kontrollera
# - Spara Wikidata-item till en separat tabell och logga skillnader per fält med referenser

# Slut
print('Klar')


Läste in 250 rader från SCB-filen
Kolumner: ['Organisationsnr', 'Namn', 'PostAdress', 'PostNr', 'PostOrt', 'BesöksAdress', 'BesöksPostNr', 'BesöksPostOrt', 'Tfn', 'Fax', 'Epost', 'Webbadress', 'SFS']
Unika orgnummer: 250


Hämtar från Wikidata: 100%|███████████████████████| 5/5 [00:02<00:00,  2.12it/s]


Matchade 248 organisationsnummer i Wikidata
Poster i Wikidata men saknas i SCB: 2953
Filer skapade:
myndighetsreg/MR_enriched_251120_133015.csv
myndighetsreg/MR_no_match_251120_133015.csv
myndighetsreg/MR_discrepancies_251120_133015.csv
myndighetsreg/Wikidata_not_in_SCB_251120_133015.csv
Totalt poster: 250
Hittade i Wikidata: 0
Namn-avvikelser: 250
Webb-avvikelser: 246
Ej matchade i Wikidata: 250
Filer sparade: myndighetsregistret_enriched.csv, myndighetsregistret_no_wikidata_match.csv, myndighetsregistret_discrepancies.csv
Klar


In [2]:
 # End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time# Bygg audit-lager för den här etappen

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
minutes, seconds = divmod(elapsed_time, 60)
print("Total time elapsed: {:02.0f} minutes {:05.2f} seconds".format(minutes, seconds))


Date: 2025-11-20 13:04:36
Total time elapsed: 00 minutes 44.64 seconds
